In [ ]:
import os
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv
from fpdf import FPDF

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


In [ ]:
def write_file(content):
    # Create the weekly_menu folder if it doesn't exist
    os.makedirs("weekly_menu", exist_ok=True)

    # Generate a timestamped filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"weekly_menu/menu_{timestamp}.md"

    # Save the menu to the file
    with open(filename, "w") as file:
        file.write(content)

        # Save the menu as a PDF

        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        for line in content.splitlines():
            pdf.multi_cell(0, 10, line)

        pdf_filename = filename.replace(".md", ".pdf")
        pdf.output(pdf_filename)

        print(f"Weekly vegetarian menu also saved to {pdf_filename}")
       

def generate_menu():
    # Prompt for generating a vegetarian weekly menu
    prompt = """
    Generate a weekly vegetarian menu with breakfast, lunch, and dinner for each day. 
    Ensure the meals are balanced and include a variety of ingredients. 
    Use seasonal vegetables and grains. It is currently Spring and user is in the UK.
    Include recipes for each meal, including cooking times and serving sizes.
    The menu should be suitable for two people.
    Output as markdown would be good
    """
    

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


# Get the generated menu
menu = generate_menu()

filename = write_file(menu)